In [12]:
## Nipype Workflow for Erin's DTI Data
### Convert RAW DICOM data to NIFTI Image Data set
import os, sys
from os.path import join as oj
from glob import glob

In [13]:
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
import nipype.interfaces.fsl as fsl

In [14]:
### This scripts assumes all of the RAW data has already been converted to NIFTI and is dumped into 
### A Single directory for each subject...
StoutHaberNIIData = '/FEATURES/NiPypeWorkingData/StoutHaber/NII_Data_Complete/'
NiPypeOutputDir = '/FEATURES/NiPypeWorkingData/StoutHaber/niPypePreProc/'
### Add additional PATHS HERE if needed

In [15]:
HomoFaberImageSessions = os.listdir(StoutHaberNIIData)

## LETS BE SMART..
StoutSubjectList = []  ### This contains a list of all the ImageSessions with a Complete Data set
for ss in HomoFaberImageSessions:
    fullSubjDirPath =  oj(StoutHaberNIIData,ss)
    if not os.path.isdir(fullSubjDirPath):
        print "UH OH!!! Directory Missing",fullSubjDirPath
    else:
        StoutSubjectList.append(ss)
print len(StoutSubjectList),"Subjects to process"

49 Subjects to process


In [16]:
### Let's create our preprocessing workflow
stoutPreProc_wf = pe.Workflow('stoutPreProc_wf') ## Initialize the workflow
stoutPreProc_wf.base_dir = NiPypeOutputDir  ## Tell it where to dump the results of the workflow

### Create the list of Subjects to Process
imageSession_InfoSrc  = pe.Node(util.IdentityInterface(fields=['imageSessionName']),name='imageSession_InfoSrc')
imageSession_InfoSrc.iterables = ('imageSessionName', HomoFaberImageSessions)

StoutDataSourceDict = dict(  
        T1 = [['imageSessionName',          'T1/struc_raw.nii.gz']],
        nodif_PA=[['imageSessionName',      'DTI/preprocess/5B0_PA/nodif_PA.nii.gz']], 
        nodif_PA_bvec=[['imageSessionName', 'DTI/preprocess/5B0_PA/nodif_PA_bvec']],
        nodif_PA_bval=[['imageSessionName', 'DTI/preprocess/5B0_PA/nodif_PA_bval']],
        data_AP=[['imageSessionName',       'DTI/preprocess/5B0_PA/nodif_PA.nii.gz']], 
        data_AP_bval=[['imageSessionName',  'DTI/preprocess/AP/data_AP_bval']],
        data_AP_bvec=[['imageSessionName',  'DTI/preprocess/AP/data_AP_bvec']])

# ## Create a datasource.. this basically helps me find the individual image files and data sets for an image session
# ## a single image directory likely consists of DTI data, T2 images, T1 images, etc, etc
datasource = pe.Node(interface=nio.DataGrabber(infields=['imageSessionName'], outfields=StoutDataSourceDict.keys() ), name='datasource') 
datasource.inputs.base_directory = StoutHaberNIIData
datasource.inputs.template = '%s/%s'
datasource.inputs.sort_filelist = True
datasource.inputs.base_directory = StoutHaberNIIData
datasource.inputs.template_args = StoutDataSourceDict

In [6]:
from nipype.interfaces.utility import Function
# get AP avg b0

### I need to get the B0 images which come out of the FSL split and select the first
### several images needed for the average
def getB0images( input_files ):
    """This will return an image stack for merging """
    import os
    b0_Images = []
    b0_image_suffixes = [ 'vol0000.nii.gz','vol0001.nii.gz','vol0002.nii.gz','vol0003.nii.gz','vol0004.nii.gz',
                          'vol0005.nii.gz','vol0006.nii.gz']
    for fwp in input_files: #file w path
        if os.path.basename(fwp) in b0_image_suffixes:
            b0_Images.append(fwp)  ###means I found one of the b0 image files I wanted
    return b0_Images  ### List of images to merge
   
getB0images_interface = Function(input_names=["input_files"], output_names=["b0_image_list"], function=getB0images)

In [ ]:
# def getDiffimages( input_files ):  
#     """This will return an image stack for merging """
#     import os
#     b0_Images = []
#     b0_image_suffixes = [ 'vol0000.nii.gz','vol0001.nii.gz','vol0002.nii.gz','vol0003.nii.gz','vol0004.nii.gz',
#                           'vol0005.nii.gz','vol0006.nii.gz']
#     for fwp in input_files: #file w path
#         if os.path.basename(fwp) in b0_image_suffixes:
#             b0_Images.append(fwp)  ###means I found one of the b0 image files I wanted
#     return b0_Images  ### List of images to merge
   
# getB0images_interface = Function(input_names=["input_files"], output_names=["b0_image_list"], function=getB0images)

In [7]:
split_AP = pe.Node(fsl.Split(dimension='t'), name='split_AP')
get_B0_images = pe.Node(interface=getB0images_interface, name='get_B0_images' )
merge_AP_B0 = pe.Node(fsl.Merge(dimension='t'), name='merge_AP_B0')
average_AP_B0 = pe.Node(fsl.MeanImage(dimension='t'), name='average_AP_B0')


In [ ]:
### Process the PA Images as well

In [8]:
stoutPreProc_wf.connect(imageSession_InfoSrc,'imageSessionName', datasource, 'imageSessionName')
stoutPreProc_wf.connect(datasource,'data_AP',split_AP,'in_file')
stoutPreProc_wf.connect(split_AP,'out_files', get_B0_images, 'input_files')
stoutPreProc_wf.connect(get_B0_images,'b0_image_list', merge_AP_B0, 'in_files')
stoutPreProc_wf.connect(merge_AP_B0,'merged_file', average_AP_B0, 'out_file')  ## Average the merged AP B0 files


#average_AP_B0 = pe.Node(fsl.MeanImage(dimension='t'), name='average_AP_B0')

#stoutPreProc_wf.connect(get_B0_images,'b0_image_list', get_B0_images, 'input_files')
#stoutPreProc_wf.connect(split_AP,'out_files', merge_AP_B0,'in_files')

In [10]:
stoutPreProc_wf.run(plugin='MultiProc', plugin_args={'n_procs' : 10})

/home/dgutman/anaconda/envs/dag_nipype/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/home/dgutman/anaconda/envs/dag_nipype/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


170111-22:12:32,729 workflow INFO:
	 ['check', 'execution', 'logging']
170111-22:12:32,835 workflow INFO:
	 Running in parallel.
170111-22:12:32,844 workflow INFO:
	 Submitting 1 jobs
170111-22:12:32,845 workflow INFO:
	 Executing: datasource ID: 0
170111-22:12:32,864 workflow INFO:
	 Executing node datasource in dir: /FEATURES/NiPypeWorkingData/StoutHaber/niPypePreProc/stoutPreProc_wf/datasource
170111-22:13:32,920 workflow ERROR:
	 ['Node datasource failed to run on host oppenheimer.']
170111-22:13:32,923 workflow INFO:
	 Saving crash info to /home/dgutman/devel/GutmanLabNipypeWorkflows/IpythonNotebooks/crash-20170111-221332-dgutman-datasource.pklz
170111-22:13:32,925 workflow INFO:
	 Traceback (most recent call last):
  File "/home/dgutman/anaconda/envs/dag_nipype/lib/python2.7/site-packages/nipype/pipeline/plugins/multiproc.py", line 18, in run_node
    result['result'] = node.run(updatehash=updatehash)
  File "/home/dgutman/anaconda/envs/dag_nipype/lib/python2.7/site-packages/nipy

RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:
MeanImage  #### Take the Mean Image

In [ ]:
# SplitME = fsl.Split(dimension='t')
# SplitME.inputs.in_file = '/FEATURES/NiPypeWorkingData/StoutHaber/NII_Data_Complete/Subj01_Scan1/DTI/preprocess/AP/data_AP.nii.gz'
# result = SplitME.run()
# print result.outputs.out_files

In [ ]:
#  http://nipype.readthedocs.io/en/latest/interfaces/generated/nipype.interfaces.fsl.maths.html#meanimage

In [ ]:

$volSTRING="07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 9
4 95 96 97";
@vol = split(/ /,$volSTRING);

$roiSTRING="IFG-vPrCG";
@roi = split(/ /,$roiSTRING);

$hemiSTRING="L R";
@hemi = split(/ /,$hemiSTRING);

# run topup first, then feed topup output into eddy (the new eddy, not the old eddy_correct)

    
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol -t ";
#print "$statement \n";
$statement  = " fslmaths $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol0000.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol0001.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol0002.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol0003.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol0004.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol0005.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol0006.nii.gz ";
$statement .= " -div 7 $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/avg_b0_AP.nii.gz ";
#print "$statement \n";

# get PA avg b0
$statement  = " fslsplit $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/5B0_PA/nodif_PA.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/5B0_PA/vol -t ";
#print "$statement \n";
$statement  = " fslmaths $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/5B0_PA/vol0000.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/5B0_PA/vol0001.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/5B0_PA/vol0002.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/5B0_PA/vol0003.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/5B0_PA/vol0004.nii.gz ";
$statement .= " -div 5 $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/5B0_PA/avg_b0_PA.nii.gz ";
#print "$statement \n";

# get AP dti
$x=0;
$statement  = " fslmerge -t $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/dti_AP.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol00" . $vol[$x] . ".nii.gz ";
for($x=1;$x<=$#vol;$x++)        {
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/vol00" . $vol[$x] . ".nii.gz ";
                                }
#print "$statement \n";
$statement  = " fslroi $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/dti_AP.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/dti_AP_trim.nii.gz ";
$statement .= " 0 -1 0 -1 0 110 ";
#print "$statement \n"; # topup needs an even # of slices

    

# make merged avg b0
$statement  = " fslmerge -t $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_AP-PA.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/avg_b0_AP.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/5B0_PA/avg_b0_PA.nii.gz ";
#print "$statement \n";
$statement  = " fslroi $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_AP-PA.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_AP-PA_trim.nii.gz ";
$statement .= " 0 -1 0 -1 0 110 ";
#print "$statement \n";  # topup needs an even # of slices
$statement  = " fslmerge -t $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_AP-PA_dti_trim.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_AP-PA_trim.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/dti_AP_trim.nii.gz ";
#print "$statement \n";  # will be input for eddy

# topup unwarping
$statement  = " topup ";
$statement .= " --imain=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_AP-PA_trim.nii.gz ";
$statement .= " --datain=$WORKINGDATAPATH/acq_params.txt ";
$statement .= " --config=/usr/share/fsl/5.0/etc/flirtsch/b02b0.cnf ";
$statement .= " --out=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/topup ";
$statement .= " --fout=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/topup_fieldmap "; # can be used for FEAT
$statement .= " --iout=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_unwarped.nii.gz ";
#print "$statement \n";
$statement  = " applytopup ";
$statement .= " --method=jac "; # because dti images are only acquired in AP direction; not merging AP+PA
$statement .= " --imain=$WORKINGDATAPATH/" . $subj[$i] . "/DTI/preprocess/AP/dti_AP_trim.nii.gz ";
$statement .= " --inindex=1 "; # because in the merged_b0_AP-PA file used to drive topup, the AP data is first
$statement .= " --datain=$WORKINGDATAPATH/acq_params.txt ";
$statement .= " --topup=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/topup ";
$statement .= " --out=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/dti_unwarped.nii.gz ";
#print "$statement \n";
    

# make data, nodif, and nodif_brain_mask
$statement  = " fslsplit $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_unwarped.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_unwarped_vol -t ";
#print "$statement \n";
$statement  = " fslmaths $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_unwarped_vol0000 ";
$statement .= " -add $WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_unwarped_vol0001 ";
$statement .= " -div 2 $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif.nii.gz ";
#print "$statement \n";
$statement  = " bet $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif_brain ";
$statement .= " -m -f 0.3 -R";
#print "$statement \n";
$statement  = " fast -B -g -t 2 -o $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain.nii.gz ";
#print "$statement \n";
$statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_seg_0.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_GM.nii.gz ";
#print "$statement \n";
$statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_seg_1.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_WM.nii.gz ";
#print "$statement \n";

# eddy
$statement  = " eddy ";
$statement .= " --imain=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/merged_b0_AP-PA_dti_trim.nii.gz ";
$statement .= " --mask=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif_brain_mask.nii.gz ";
$statement .= " --index=$WORKINGDATAPATH/eddy_index.txt ";
$statement .= " --acqp=$WORKINGDATAPATH/acq_params.txt ";
$statement .= " --topup=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/preprocess/topup ";
$statement .= " --bvecs=$WORKINGDATAPATH/eddy_bvecs ";
$statement .= " --bvals=$WORKINGDATAPATH/eddy_bvals ";
$statement .= " --out=$WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data_eddy ";
#print "$statement \n";
$statement  = " fslroi ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data_eddy.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif1.nii.gz ";
$statement .= " 0 1 ";
#print "$statement \n";
$statement  = " fslroi ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data_eddy.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif2.nii.gz ";
    
$statement .= " 1 1 ";
#print "$statement \n";
$statement  = " fslmaths $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif1.nii.gz ";
$statement .= " -add $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif2.nii.gz ";
$statement .= " -div 2 $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif.nii.gz ";
#print "$statement \n";
$statement  = " fslroi ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data_eddy.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data_dti.nii.gz ";
$statement .= " 2 91 ";
#print "$statement \n";
$statement  = " fslmerge -t $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif.nii.gz ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data_dti.nii.gz ";
#print "$statement \n";
    # dtifit
$statement  = " dtifit ";
$statement .= " -k $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data.nii.gz ";
$statement .= " -o $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/data ";
$statement .= " -m $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/nodif_brain_mask.nii.gz ";
$statement .= " -r $WORKINGDATAPATH/bvecs ";
$statement .= " -b $WORKINGDATAPATH/bvals ";
#print "$statement \n";

# bedpost
$statement  = " cp $WORKINGDATAPATH/bvecs ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/bvecs ";
#print "$statement \n";
$statement  = " cp $WORKINGDATAPATH/bvals ";
$statement .= " $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/bvals ";
#print "$statement \n";
$statement  = " bedpostx $WORKINGDATAPATH/" .$subj[$i] . "/DTI/data/";
#print "$statement \n";

# T1 preprocessing
$statement  = " fslreorient2std $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_raw.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_raw.nii.gz ";
#print "$statement \n";
$statement  = " bet $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_raw.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain ";
$statement .= " -m -f 0.4 -g -0.1 -B ";
#print "$statement \n";
$statement  = " fast -B -g -o $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain.nii.gz ";
#print "$statement \n";
$statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain_restore.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain.nii.gz ";
#print "$statement \n";
$statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain_seg_1.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain_GM.nii.gz ";
#print "$statement \n";
$statement  = " mv $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain_seg_2.nii.gz ";
$statement .= " $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain_WM.nii.gz ";
#print "$statement \n";
    

## REGISTRATION

$statement  = " mkdir $WORKINGDATAPATH/" . $subj[$i] . "/xfms/ ";
#print "$statement \n";

$statement  = " flirt -dof 12 -in $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain.nii.gz ";
$statement .= " -ref $WORKINGDATAPATH/MNI152_T1_1mm_brain.nii.gz ";
$statement .= " -omat $WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_12dof_MNI.mat ";
#print "$statement \n";

$statement  = " fnirt --in=$WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_raw.nii.gz ";
$statement .= " --ref=$WORKINGDATAPATH/MNI152_T1_1mm.nii.gz ";
$statement .= " --aff=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_12dof_MNI.mat ";
$statement .= " --cout=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_warp_MNI_warpfield.nii.gz ";
#print "$statement \n";

$statement  = " applywarp --in=$WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain.nii.gz ";
$statement .= " --ref=$WORKINGDATAPATH/MNI152_T1_1mm.nii.gz ";
$statement .= " --warp=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_warp_MNI_warpfield.nii.gz ";
$statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_2_MNI.nii.gz ";
#print "$statement \n";

$statement  = " invwarp --warp=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_warp_MNI_warpfield.nii.gz ";
$statement .= " --ref=$WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain.nii.gz ";
$statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_warpfield.nii.gz ";
#print "$statement \n";

$statement  = " flirt -dof 12 -in $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain.nii.gz ";
$statement .= " -ref $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain.nii.gz ";
$statement .= " -omat $WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc.mat ";
#print "$statement \n";

$statement  = " flirt -in $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain.nii.gz ";
$statement .= " -ref $WORKINGDATAPATH/" . $subj[$i] . "/T1/struc_brain.nii.gz ";
$statement .= " -applyxfm -init $WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc.mat ";
$statement .= " -out $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_2_struc.nii.gz ";
#print "$statement \n";

$statement  = " applywarp --in=$WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain.nii.gz ";
$statement .= " --ref=$WORKINGDATAPATH/MNI152_T1_1mm.nii.gz ";
$statement .= " --premat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc.mat ";

    $statement .= " --warp=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_warp_MNI_warpfield.nii.gz ";
$statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_2_MNI.nii.gz ";
#print "$statement \n";

$statement  = " convert_xfm ";
$statement .= " -omat $WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_12dof_nodif.mat ";
$statement .= " -inverse $WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc.mat ";
#print "$statement \n";

$statement  = " convertwarp ";
$statement .= " --ref=$WORKINGDATAPATH/MNI152_T1_1mm_brain.nii.gz ";
$statement .= " --premat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc.mat ";
$statement .= " --warp1=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_warp_MNI_warpfield.nii.gz ";
$statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc_warp_MNI_warpfield.nii.gz ";
print "$statement \n";

$statement  = " convertwarp ";
$statement .= " --ref=$WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_mask.nii.gz ";
$statement .= " --warp1=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_warpfield.nii.gz ";
$statement .= " --postmat=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/struc_12dof_nodif.mat ";
$statement .= " --out=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_12dof_nodif_warpfield.nii.gz ";
print "$statement \n";

$statement  = " probtrackx2 --pd --onewaycondition --omatrix2 ";
$statement .= " -s $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data.bedpostX/merged ";
$statement .= " -m $WORKINGDATAPATH/" . $subj[$i] . "/DTI/data/nodif_brain_mask.nii.gz ";
$statement .= " -x $WORKINGDATAPATH/ROIs/Human_" . $roi[$r] . "_" . $hemi[$h] . ".nii.gz ";
$statement .= " --target2=$WORKINGDATAPATH/MNI152_T1_1mm_brain_mask_downsample_2.nii.gz ";
$statement .= " -l -c 0.2 -S 2000 --steplength=0.5 -P 5000 --fibthresh=0.1 --randfib=0 ";  # check samples
$statement .= " --xfm=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/MNI_warp_struc_12dof_nodif_warpfield.nii.gz ";
$statement .= " --invxfm=$WORKINGDATAPATH/" . $subj[$i] . "/xfms/nodif_12dof_struc_warp_MNI_warpfield.nii.gz ";
$statement .= " --forcedir --opd ";
$statement .= " --dir=$OUTPUTDATAPATH/" . $subj[$i] . "_" . $roi[$r] . "_" . $hemi[$h] . "_Segmentation  ";
print "$statement \n";

}